In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 973.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [-30, -31.02],
	"longitude": [140, 130],
	"start_date": "2019-08-01",
	"end_date": "2020-01-11",
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "weather_code", "surface_pressure", "wind_speed_10m", "wind_direction_10m", "wind_direction_100m", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(4).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(5).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_direction_100m = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_temperature_7_to_28cm = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_temperature_28_to_100cm = hourly.Variables(10).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["weather_code"] = hourly_weather_code
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

In [ ]:
'''
Generate Australia coordinates
'''
'''
origin:
33x42x100x100x24x365 (year) = 121,413,600,000

less Shrink: scale 0.01
10x10x100x100x24x1 = 24,000,000

shrink: scale 0.1
10x10x10x10x24x1 = 240,000
'''
def generate_all_australia_coordinates():
    # Australia latitude range: -43 to -10
    lat_min, lat_max = -38.9, -37.0
    # Australia longitude range: 112 to 154
    lon_min, lon_max = 149.0, 150.0

    #coordinates = []
    longitudes = []
    latitudes = []

    # Iterate through latitudes and longitudes in steps of 0.0001

    lat = lat_min
    while lat <= lat_max:
        lon = lon_min
        while lon <= lon_max:
            longitudes.append(round(lon, 1))
            latitudes.append(round(lat, 1))
            lon += 0.1
        lat += 0.1


    #return coordinates
    return longitudes, latitudes

In [ ]:
longitudes, latitudes = generate_all_australia_coordinates()

In [ ]:
len(longitudes)

Connect to Open-Meteo API

In [ ]:

'''
API SET UP
[hourly]: are the weather attributes we may want to use
'''


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": latitudes,
	"longitude": longitudes,
	"start_date": "2019-08-01",
	"end_date": "2020-01-11",
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "weather_code", "surface_pressure", "wind_speed_10m", "wind_direction_10m", "wind_direction_100m", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm"]
}
responses = openmeteo.weather_api(url, params=params)

In [ ]:
'''
Generate a dataframe store each coordinate's historical data
'''


def get_weather_data(coordinate):
  hourly = response.Hourly()
  hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
  hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
  hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
  hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
  hourly_weather_code = hourly.Variables(4).ValuesAsNumpy()
  hourly_surface_pressure = hourly.Variables(5).ValuesAsNumpy()
  hourly_wind_speed_10m = hourly.Variables(6).ValuesAsNumpy()
  hourly_wind_direction_10m = hourly.Variables(7).ValuesAsNumpy()
  hourly_wind_direction_100m = hourly.Variables(8).ValuesAsNumpy()
  hourly_soil_temperature_7_to_28cm = hourly.Variables(9).ValuesAsNumpy()
  hourly_soil_temperature_28_to_100cm = hourly.Variables(10).ValuesAsNumpy()

  hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
  )}
  hourly_data["latitude"] = coordinate.Latitude()
  hourly_data["longitude"] = coordinate.Longitude()
  hourly_data["temperature_2m"] = hourly_temperature_2m
  hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
  hourly_data["dew_point_2m"] = hourly_dew_point_2m
  hourly_data["precipitation"] = hourly_precipitation
  hourly_data["weather_code"] = hourly_weather_code
  hourly_data["surface_pressure"] = hourly_surface_pressure
  hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
  hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
  hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
  hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
  hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm

  return hourly_data

In [ ]:
def expected_variables(coordinates):
    #print(coordinates)
    total_df = pd.DataFrame()
    coordinate_checking = {}
    for coordinate in coordinates:

        api_lat = coordinate.Latitude()
        api_lon = coordinate.Longitude()
        api_coord = (api_lat, api_lon)

        if api_coord not in coordinate_checking:

            coordinate_checking[api_coord] = 1

            # Process hourly data. The order of variables needs to be the same as requested.
            hourly_data = get_weather_data(coordinate)
            hourly_df = pd.DataFrame(data = hourly_data)

            total_df = pd.concat([total_df, hourly_df], ignore_index=True)
        else:
            continue

    return total_df

In [ ]:
results = expected_variables(responses)

In [ ]:
results.to_csv('4139_149150_data.csv', index = False)